In [ ]:
%pip install yfinance

In [ ]:
%pip install tabulate

In [ ]:
import yfinance as yf

# 테슬라 (TSLA)에 대한 Ticker 객체 생성
tsla = yf.Ticker("TSLA")

# Ticker 객체에 대한 정보 출력 (.py에서 실행할 때는 print(tsla.info)로 사용)
tsla.info

In [ ]:
hist = tsla.history(period="5d") # 5일간의 주가 데이터를 가져옴
display(hist) # 데이터 출력

In [ ]:
tsla.recommendations # 추천 정보 출력

In [ ]:
from datetime import datetime
import pytz
import yfinance as yf
import tabulate

In [ ]:
def get_current_time(timezone: str = 'Asia/Seoul'):
    tz = pytz.timezone(timezone) # 타임존 설정
    now = datetime.now(tz).strftime("%Y-%m-%d %H:%M:%S")
    now_timezone = f'{now} {timezone}'
    print(now_timezone)
    return now_timezone

In [ ]:
def get_yf_stock_info(ticker: str):
    stock = yf.Ticker(ticker)
    info = stock.info
    print(info)
    return str(info)

def get_yf_stock_history(ticker: str, period: str):
    stock = yf.Ticker(ticker)
    history = stock.history(period=period)
    history_md = history.to_markdown() # 데이터프레임을 마크다운 형식으로 변환
    print(history_md)
    return history_md

def get_yf_stock_recommendations(ticker: str):
    stock = yf.Ticker(ticker)
    recommendations = stock.recommendations
    recommendations_md = recommendations.to_markdown() # 데이터프레임을 마크다운 형식으로 변환
    print(recommendations_md)
    return recommendations_md

In [ ]:
tools = [
    {
        "type": "function",
        "function": {
            "name": "get_current_time",
            "description": "해당 타임존의 날짜와 시간을 반환합니다.",
            "parameters": {
                "type": "object",
                "properties": {
                    'timezone': {
                        'type': 'string',
                        'description': '현재 날짜와 시간을 반환할 타임존을 입력하세요. (예: Asia/Seoul)',
                    },
                },
                "required": ['timezone'],
            },
        }
    },
    {
        "type": "function",
        "function": {
            "name": "get_yf_stock_info",
            "description": "해당 종목의 Yahoo Finance 정보를 반환합니다.",
            "parameters": {
                "type": "object",
                "properties": {
                    'ticker': {
                        'type': 'string',
                        'description': 'Yahoo Finance 정보를 반환할 종목의 티커를 입력하세요. (예: AAPL)',
                    },
                },
                "required": ['ticker'],
            },
        }
    },
    {
        "type": "function",
        "function": {
            "name": "get_yf_stock_history",
            "description": "해당 종목의 Yahoo Finance 주가 정보를 반환합니다.",
            "parameters": {
                "type": "object",
                "properties": {
                    'ticker': {
                        'type': 'string',
                        'description': 'Yahoo Finance 주가 정보를 반환할 종목의 티커를 입력하세요. (예: AAPL)',
                    },
                    'period': {
                        'type': 'string',
                        'description': '주가 정보를 조회할 기간을 입력하세요. (예: 1d, 5d, 1mo, 1y, 5y)',
                    },
                },
                "required": ['ticker', 'period'],
            },
        }
    },
    {
        "type": "function",
        "function": {
            "name": "get_yf_stock_recommendations",
            "description": "해당 종목의 Yahoo Finance 추천 정보를 반환합니다.",
            "parameters": {
                "type": "object",
                "properties": {
                    'ticker': {
                        'type': 'string',
                        'description': 'Yahoo Finance 추천 정보를 반환할 종목의 티커를 입력하세요. (예: AAPL)',
                    },
                },
                "required": ['ticker'],
            },
        }
    },
]

In [ ]:
get_yf_stock_history('AAPL', '5d') # 애플 주가 정보
print('----')
get_yf_stock_recommendations('AAPL')

In [ ]:
from openai import OpenAI
from dotenv import load_dotenv
import os
import json
# import streamlit as st

In [ ]:
load_dotenv()
api_key = os.getenv("OPEN_API_KEY")
client = OpenAI(api_key=api_key)  # OpenAI 클라이언트 인스턴스 생성

In [ ]:
def get_ai_response(messages, tools=None):
    response = client.chat.completions.create(
        model="gpt-4o",  # 사용할 모델 지정
        messages=messages,  # 대화 기록 전달
        tools=tools,  # 사용 가능한 도구 전달
    )
    return response  # 생성된 응답 반환

In [ ]:
# 초기 메시지 설정
messages = [
    {"role": "system", "content": "너는 사용자를 도와주는 상담사야."},  # 시스템 메시지
]

print("GPT와 대화를 시작합니다. 'exit'을 입력하면 종료됩니다.")

while True:
    # 사용자 입력 받기
    user_input = input("사용자: ")
    if user_input.lower() == "exit":  # 'exit' 입력 시 종료
        print("대화를 종료합니다.")
        break

    # 사용자 메시지를 대화 기록에 추가
    messages.append({"role": "user", "content": user_input})

    # GPT 응답 생성
    ai_response = get_ai_response(messages, tools=tools)
    ai_message = ai_response.choices[0].message
    print(f"GPT: {ai_message.content}")  # GPT 응답 출력

    # 도구 호출 처리
    tool_calls = ai_message.tool_calls  # AI 응답에 포함된 tool_calls 가져오기
    if tool_calls:  # tool_calls가 있는 경우
        for tool_call in tool_calls:
            tool_name = tool_call.function.name  # 호출할 함수 이름
            tool_call_id = tool_call.id  # tool_call ID
            arguments = json.loads(tool_call.function.arguments)  # 매개변수 파싱

            # 호출할 함수 실행
            if tool_name == "get_current_time":
                func_result = get_current_time(timezone=arguments["timezone"])
            elif tool_name == "get_yf_stock_info":
                func_result = get_yf_stock_info(ticker=arguments["ticker"])
            elif tool_name == "get_yf_stock_history":
                func_result = get_yf_stock_history(
                    ticker=arguments["ticker"], period=arguments["period"]
                )
            elif tool_name == "get_yf_stock_recommendations":
                func_result = get_yf_stock_recommendations(ticker=arguments["ticker"])
            else:
                func_result = "알 수 없는 도구 호출입니다."

            # 함수 실행 결과를 대화 기록에 추가
            messages.append(
                {
                    "role": "function",
                    "tool_call_id": tool_call_id,
                    "name": tool_name,
                    "content": func_result,
                }
            )

        # 도구 호출 결과를 바탕으로 GPT 응답 생성
        messages.append(
            {"role": "system", "content": "이제 주어진 결과를 바탕으로 답변할 차례다."}
        )
        ai_response = get_ai_response(messages, tools=tools)
        ai_message = ai_response.choices[0].message

        # 최종 GPT 응답 출력
        print(f"GPT: {ai_message.content}")
        messages.append({"role": "assistant", "content": ai_message.content})